## Nth Largest

-- use cases include 0 and -1

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
%load_ext sql
%sql postgresql://toofanmacpro@localhost:5432/testdb


In [3]:
def fromDB(query: str) -> pd.DataFrame:
    try:
        engine = create_engine('postgresql://toofanmacpro@localhost:5432/testdb')
        try:
            df = pd.read_sql(query, engine)
            return df
        except Exception as e:
            print(f"extraction failed: {e}")
    except Exception as e:
        print(f"connection failure: {e}")

In [4]:
%%sql
SELECT * FROM "Employee";

 * postgresql://toofanmacpro@localhost:5432/testdb
3 rows affected.


index,id,salary
0,1,100
1,2,200
2,3,300


In [5]:
%%sql
CREATE OR REPLACE FUNCTION NthLargest (N Int) RETURNS TABLE (salary BIGINT) AS
$$
BEGIN
    RETURN QUERY(
        SELECT 
            CASE WHEN N <= 0 THEN NULL
            ELSE
                (SELECT 
                    DISTINCT e.salary
                FROM "Employee" AS e
                ORDER BY e.salary DESC
                LIMIT 1 OFFSET (N-1)) END AS salary
    );
END;
$$ LANGUAGE plpgsql

 * postgresql://toofanmacpro@localhost:5432/testdb
Done.


[]

In [6]:
%%sql
SELECT NthLargest(2)

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


nthlargest
200


In [7]:
%%sql
SELECT NthLargest(0)

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


nthlargest
None


In [8]:
%%sql
SELECT NthLargest(-1)

 * postgresql://toofanmacpro@localhost:5432/testdb
1 rows affected.


nthlargest
None


In [9]:
query = 'SELECT * FROM "Employee";'

Employee = fromDB(query)

Employee

,index,id,salary
0,0,1,100
1,1,2,200
2,2,3,300


### My Pandas solution

In [10]:
id = [1,2,3]
salary = [100,200,300]

df = pd.DataFrame({'id':id, 'salary': salary})

In [15]:
def nLargest(daa: pd.DataFrame, N: int) -> pd.DataFrame: # '->' indicates what the output should be, and ':' defines what the inputs should be
    if N <= 0 : 
        return (pd.DataFrame({"SecondHighestSalary": [None]}, index = [0]))

    unique = (daa
          .salary
          .drop_duplicates()
          .nlargest(N)                                          # nlargest() gets you results in descending order
          )

    print(unique)
    
    if (len(unique) < N):
        return (pd.DataFrame({"SecondHighestSalary": [None]}, index = [0]))
    else:
        return (pd.DataFrame({"SecondHighestSalary":[unique.iloc[-1]]}, index = [0]))

In [16]:
nLargest(df,2)

2    300
1    200
Name: salary, dtype: int64


,SecondHighestSalary
0,200


In [17]:
nLargest(df,0)

,SecondHighestSalary
0,None


In [18]:
nLargest(df,-10)

,SecondHighestSalary
0,None


###  MY SQL Query of NthLargest

In [ ]:
%%sql
SELECT 
    DISTINCT salary
FROM "Employee"
ORDER BY salary DESC
LIMIT 1 OFFSET 1;

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION NthLargest(N INT) RETURNS TABLE (salary BIGINT) AS $$
BEGIN
RETURN QUERY (
    SELECT
        CASE WHEN N <= 0 THEN NULL
        ELSE 
            (
                SELECT 
                    DISTINCT e.salary
                FROM "Employee" AS e
                ORDER BY e.salary DESC
                LIMIT 1 OFFSET (N-1)
            )
        END AS salary
    );
END;
$$ LANGUAGE plpgsql;

In [ ]:
%%sql
SELECT * FROM NthLargest(1)

IF a funciton has to be dropped

In [ ]:
# %%sql
# DROP FUNCTION nthlargest(integer)

In [ ]:
%%sql
SELECT * FROM NthLargest(-1)

### Nth Highest = SQL - best solution

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION NthBigSalary(N INT) RETURNS TABLE(salary BIGINT) AS $$
BEGIN
    IF N < 1 THEN 
        RETURN QUERY (SELECT NULL::BIGINT AS salary);
    ELSE
        RETURN QUERY(
            SELECT 
                e.salary
            FROM "Employee" AS e
            GROUP BY e.salary
            ORDER BY e.salary DESC
            OFFSET N-1 LIMIT 1
        );
    END IF;
END;
$$ LANGUAGE plpgsql;

we used BIGINT because thats how our data is stored

In [ ]:
%%sql
SELECT * FROM NthBigSalary(2)

In [ ]:
%%sql
SELECT * FROM NthBigSalary(-1)

In [ ]:
%%sql
SELECT * FROM NthBigSalary(0)